## Problem D

The following MIP assigns students into project teams to balance the overall characteristics of each team.

**Data:**

- $I$: set of students.
- $n$: number of teams
- $J=\{1,2,\cdots,n\}$ : set of teams.
- $K$: set of characteristics.
- $a_{ik}$: student $i$'s value for characteristics $k$.
- $w_k$: the weight for characteristics $k$ in the objective.
- $L_k$: the ideal lower bound for the sum of characteristic $k$ for any team. 
- $U_k$: the ideal upper bound for the sum of characteristics $k$ for any team.

You should assume that the data is given in a excel file with the same format as the `PS10-Team-input-1.xlsx` and `PS10-Team-input-2.xlsx` files attached to this assignment. 

The sheet named "Students" encodes $I$, $K$ and $a_{ik}$'s. In the below screenshot of `PS10-Team-input-1.xlsx`, $I=\{A,B,C,D,E,F\}$, and $K=\{Person, Male, Programmer, Math, Speaking\}$.

![](PS10-Team1.png)


The sheet named "Parameters" encodes the $w_k$, $L_k$ and $U_k$ for each characteristic $k$.

![](PS10-Team2.png)

**Decision variables:**

- $x_{ij}$ : whether to assign student $i$ to team $j$. (Binary)
- $s_k$ : maximum deviation below the ideal lower bound $L_k$ for characteristic $k$. (Continuous)
- $t_k$ : maximum deviation above the ideal upper bound $U_k$ for characteristic $k$. (Continuous)

**Objective and constraints:**

$$\begin{aligned}
\text{Minimize:} && \sum_{k \in K} w_k(s_k+t_k) \\
\text{subject to:} && \\
\text{(Every person assigned)} && \sum_{j \in J} x_{ij} & = 1 && \text{For each person $i \in I$.}\\
\text{(Team balance)} && L_k - s_k \le \sum_{i \in I} a_{ik}x_{ij} & \le U_k + t_k && \text{For each team $j \in J$ and each $k \in K$.} \\
\text{(Non-negativity)} && s_k, t_k & \ge 0 && \text{for all $k$.}
\end{aligned}$$

**Write a function called "assignTeams" with the following input arguments:**

- **inputFile:** path to the input spreadsheet.
- **n:** the number of teams to divide students into.

**The function should return two variables:**

- **df:** a DataFrame with one column called "Team". The index should be the name of each individual, and the column "Team" should specify the number $j$ to which the person is assigned.
- **objval:** the optimal objective value.

For the test runs, you should download the input files attached to this exercise into the same directory as the Jupyter notebook.

In [1]:
import pandas as pd
from gurobi import Model, GRB

In [41]:
inputFile = 'PS10-Team-input-1.xlsx'
n = 2

student = pd.read_excel(inputFile, sheet_name = 'Students',index_col = 0)
parameter = pd.read_excel(inputFile, sheet_name = 'Parameters',index_col = 0)
K = parameter.columns
J = range(1,n+1)
I = student.index
W = parameter.loc['Weights',:]

mod = Model()
x = mod.addVars(I,J, name = 'x', vtype = GRB.BINARY)
s = mod.addVars(K, name = 's')
t = mod.addVars(K, name = 't')
mod.setObjective(sum(W[k]*(s[k]+t[k]) for k in K), sense = GRB.MINIMIZE)
mod.addConstrs((sum(x[i,j] for j in J) == 1 for i in I), name = 'TeamAssigned')
mod.addConstrs((sum(student.loc[i,k]*x[i,j] for i in I) >= (parameter.loc['L',k] - s[k]) for j in J for k in K))
mod.addConstrs((sum(student.loc[i,k]*x[i,j] for i in I) <= (parameter.loc['U',k] + t[k]) for j in J for k in K))

# mod.update()
# mod.write('ProblemD.lp')
# %cat 'ProblemD.lp'

mod.setParam('OutputFlag',False)
mod.optimize()
objval = mod.ObjVal

df = pd.DataFrame(index = I,columns=['Team'])
for i in I:
    for j in J:
        if x[i,j].x == 1:
            df.loc[i,'Team'] = j

df

,Team
Names,
A,2
B,2
C,2
D,1
E,1
F,1


In [43]:
# Write your final code here
def assignTeams(inputFile, n):
    student = pd.read_excel(inputFile, sheet_name = 'Students',index_col = 0)
    parameter = pd.read_excel(inputFile, sheet_name = 'Parameters',index_col = 0)
    K = parameter.columns
    J = range(1,n+1)
    I = student.index
    W = parameter.loc['Weights',:]

    mod = Model()
    x = mod.addVars(I,J, name = 'x', vtype = GRB.BINARY)
    s = mod.addVars(K, name = 's')
    t = mod.addVars(K, name = 't')
    mod.setObjective(sum(W[k]*(s[k]+t[k]) for k in K), sense = GRB.MINIMIZE)
    mod.addConstrs((sum(x[i,j] for j in J) == 1 for i in I), name = 'TeamAssigned')
    mod.addConstrs((sum(student.loc[i,k]*x[i,j] for i in I) >= (parameter.loc['L',k] - s[k]) for j in J for k in K))
    mod.addConstrs((sum(student.loc[i,k]*x[i,j] for i in I) <= (parameter.loc['U',k] + t[k]) for j in J for k in K))

    # mod.update()
    # mod.write('ProblemD.lp')
    # %cat 'ProblemD.lp'

    mod.setParam('OutputFlag',False)
    mod.optimize()
    objval = mod.ObjVal

    df = pd.DataFrame(index = I,columns=['Team'])
    for i in I:
        for j in J:
            if x[i,j].x == 1:
                df.loc[i,'Team'] = j

    return df, objval

In [6]:
# Test run 1
# It is okay if your team numbers are different from what's below, as there are multiple optimal solutions
df,objval=assignTeams('PS10-Team-input-1.xlsx',2)
print('Optimal objective value:',objval)
df

Optimal objective value: 0.0


,Team
Names,
A,2
B,2
C,2
D,1
E,1
F,1


In [47]:
# Test run 2
df,objval=assignTeams('PS10-Team-input-2.xlsx',10)
print('Optimal objective value:',objval)
df.sort_values(by='Team')

Optimal objective value: 8.0


,Team
Names,
XingZhou,1
Shawn,1
Xenna,1
Patty,1
Yingying,1
Ran,2
Qinan,2
Wing,2
Bob,2


In [7]:
# Test run 2
# It is okay if your team numbers are different from what's below, as there are multiple optimal solutions
df,objval=assignTeams('PS10-Team-input-2.xlsx',10)
print('Optimal objective value:',objval)
df.sort_values(by='Team')

Optimal objective value: 8.0


,Team
Names,
Bob,1
Xenna,1
Patty,1
I-Ting,1
Yingying,1
XingZhou,2
Ouyang,2
Qinan,2
Mei,2
